In [15]:
import psycopg2
from psycopg2.extras import execute_values
import yfinance as yf
import time
import numpy as np
# Database connection (replace with your credentials)
conn = psycopg2.connect(
    dbname="stock_app",
    user="postgres",
    password="nithin",
    host="localhost",
    port="5432"
)
cur = conn.cursor()


In [16]:

# Market list (3 indices)
MARKETS = ['^GSPC', '^DJI', '^IXIC']

# Helper function to safely convert to integer with debug
def safe_int(value, field_name, stock, date):
    try:
        if value is None or (isinstance(value, float) and np.isnan(value)):
            print(f"Debug: '{field_name}' for {stock} on {date} is NaN or None, using NULL")
            return None  # Maps to SQL NULL
        return int(value)
    except Exception as e:
        print(f"Debug: Error converting '{field_name}' for {stock} on {date}: {value} - {e}")
        raise

# Step 1: Create and Populate `users`
def create_users_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS users (
            user_id VARCHAR PRIMARY KEY,
            name VARCHAR NOT NULL,
            email VARCHAR UNIQUE NOT NULL
        );
    """
    cur.execute(create_sql)
    print("Table 'users' created or already exists.")

def populate_users():
    users_data = [
        ('user1', 'Alice', 'alice@example.com'),
        ('user2', 'Bob', 'bob@example.com'),
        ('user3', 'Charlie', 'charlie@example.com')
    ]
    execute_values(cur, """
        INSERT INTO users (user_id, name, email)
        VALUES %s
        ON CONFLICT (user_id) DO NOTHING
    """, users_data)
    print("Populated 'users' table.")

def verify_users():
    cur.execute("SELECT COUNT(*) FROM users;")
    count = cur.fetchone()[0]
    print(f"Verification: 'users' has {count} rows (expected 3).")
    
    cur.execute("SELECT * FROM users LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'users': {sample}")

# Step 2: Create and Populate `market_indices`
def create_market_indices_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS market_indices (
            index_symbol VARCHAR PRIMARY KEY,
            name VARCHAR NOT NULL
        );
    """
    cur.execute(create_sql)
    print("Table 'market_indices' created or already exists.")

def populate_market_indices():
    markets_data = [
        ('^GSPC', 'S&P 500'),
        ('^DJI', 'Dow Jones Industrial Average'),
        ('^IXIC', 'NASDAQ Composite')
    ]
    execute_values(cur, """
        INSERT INTO market_indices (index_symbol, name)
        VALUES %s
        ON CONFLICT (index_symbol) DO NOTHING
    """, markets_data)
    print("Populated 'market_indices' table.")

def verify_market_indices():
    cur.execute("SELECT COUNT(*) FROM market_indices;")
    count = cur.fetchone()[0]
    print(f"Verification: 'market_indices' has {count} rows (expected 3).")
    
    cur.execute("SELECT * FROM market_indices LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'market_indices': {sample}")


In [17]:
STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'JPM', 'V', 'WMT', 'DIS', 'NFLX', 'BA', 'KO', 'PEP']

# Create table
def create_sectors_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS sectors (
            sector VARCHAR PRIMARY KEY,
            name VARCHAR NOT NULL,
            overview TEXT
        );
    """
    cur.execute(create_sql)
    print("Table 'sectors' created or already exists.")

# Populate table
def populate_sectors():
    sectors_data = set()
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        info = ticker.info
        sector = info.get("sector", "Unknown")
        # Use sector as name; overview as placeholder
        sectors_data.add((sector, sector, "Sector overview TBD"))
    execute_values(cur, """
        INSERT INTO sectors (sector, name, overview)
        VALUES %s
        ON CONFLICT (sector) DO NOTHING
    """, list(sectors_data))
    print("Populated 'sectors' table.")

# Verify table
def verify_sectors():
    cur.execute("SELECT COUNT(*) FROM sectors;")
    count = cur.fetchone()[0]
    print(f"Verification: 'sectors' has {count} rows (expected ~7-10 unique sectors from 15 stocks).")
    
    cur.execute("SELECT * FROM sectors LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'sectors': {sample}")

In [18]:
STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'NVDA', 'META', 'JPM', 'V', 'WMT', 'DIS', 'NFLX', 'BA', 'KO', 'PEP']

# Create table
def create_industries_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS industries (
            industry VARCHAR PRIMARY KEY,
            sector VARCHAR NOT NULL,
            name VARCHAR NOT NULL,
            overview TEXT,
            FOREIGN KEY (sector) REFERENCES sectors(sector)
        );
    """
    cur.execute(create_sql)
    print("Table 'industries' created or already exists.")

# Populate table
def populate_industries():
    industries_data = set()
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        info = ticker.info
        sector = info.get("sector", "Unknown")
        industry = info.get("industry", "Unknown")
        # Use industry as name; overview as placeholder
        industries_data.add((industry, sector, industry, "Industry overview TBD"))
    execute_values(cur, """
        INSERT INTO industries (industry, sector, name, overview)
        VALUES %s
        ON CONFLICT (industry) DO NOTHING
    """, list(industries_data))
    print("Populated 'industries' table.")

# Verify table
def verify_industries():
    cur.execute("SELECT COUNT(*) FROM industries;")
    count = cur.fetchone()[0]
    print(f"Verification: 'industries' has {count} rows (expected ~10-15 unique industries from 15 stocks).")
    
    cur.execute("SELECT * FROM industries LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'industries': {sample}")

In [19]:
def create_stocks_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS stocks (
            ticker VARCHAR PRIMARY KEY,
            short_name VARCHAR,
            long_name VARCHAR,
            sector VARCHAR,
            industry VARCHAR,
            country VARCHAR,
            website VARCHAR,
            long_business_summary TEXT,
            market_cap BIGINT,
            current_price DECIMAL,
            change_percent DECIMAL,
            fifty_two_week_low DECIMAL,
            fifty_two_week_high DECIMAL,
            beta DECIMAL,
            trailing_pe DECIMAL,
            dividend_yield DECIMAL,
            forward_pe DECIMAL,
            price_to_book DECIMAL,
            institutional_ownership_pct DECIMAL,
            currency VARCHAR,
            FOREIGN KEY (sector) REFERENCES sectors(sector),
            FOREIGN KEY (industry) REFERENCES industries(industry)
        );
    """
    cur.execute(create_sql)
    print("Table 'stocks' created or already exists.")

# Populate table
def populate_stocks():
    stocks_data = []
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        info = ticker.info
        stocks_data.append((
            info.get("symbol", stock), info.get("shortName", "N/A"), info.get("longName", "N/A"),
            info.get("sector", "Unknown"), info.get("industry", "Unknown"), info.get("country", "N/A"),
            info.get("website", "N/A"), info.get("longBusinessSummary", "N/A"), info.get("marketCap", 0),
            info.get("regularMarketPrice", 0.0), info.get("regularMarketChangePercent", 0.0),
            info.get("fiftyTwoWeekLow", 0.0), info.get("fiftyTwoWeekHigh", 0.0), info.get("beta", 0.0),
            info.get("trailingPE", 0.0), info.get("dividendYield", 0.0), info.get("forwardPE", 0.0),
            info.get("priceToBook", 0.0), info.get("institutionalOwnership", 0.0), info.get("currency", "USD")
        ))
        time.sleep(1)  # Avoid yfinance rate limits
    execute_values(cur, """
        INSERT INTO stocks (ticker, short_name, long_name, sector, industry, country, website, long_business_summary,
        market_cap, current_price, change_percent, fifty_two_week_low, fifty_two_week_high, beta, trailing_pe,
        dividend_yield, forward_pe, price_to_book, institutional_ownership_pct, currency)
        VALUES %s
        ON CONFLICT (ticker) DO NOTHING
    """, stocks_data)
    print("Populated 'stocks' table.")

# Verify table
def verify_stocks():
    cur.execute("SELECT COUNT(*) FROM stocks;")
    count = cur.fetchone()[0]
    print(f"Verification: 'stocks' has {count} rows (expected 15).")
    
    cur.execute("SELECT ticker, short_name, sector FROM stocks LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'stocks': {sample}")

In [20]:
def create_stock_history_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS stock_history (
            ticker VARCHAR,
            date DATE,
            open DECIMAL,
            high DECIMAL,
            low DECIMAL,
            close DECIMAL,
            volume BIGINT,
            currency VARCHAR,
            PRIMARY KEY (ticker, date),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'stock_history' created or already exists.")

# Populate table
def populate_stock_history():
    history_data = []
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        hist = ticker.history(period="5y")  # 5 years of daily data
        for date, row in hist.iterrows():
            history_data.append((
                stock, date.strftime('%Y-%m-%d'),
                float(row["Open"]), float(row["High"]), float(row["Low"]), float(row["Close"]),
                int(row["Volume"]), ticker.info["currency"]
            ))
        time.sleep(1)  # Avoid rate limits
    execute_values(cur, """
        INSERT INTO stock_history (ticker, date, open, high, low, close, volume, currency)
        VALUES %s
        ON CONFLICT (ticker, date) DO NOTHING
    """, history_data)
    print("Populated 'stock_history' table.")

# Verify table
def verify_stock_history():
    cur.execute("SELECT COUNT(*) FROM stock_history;")
    count = cur.fetchone()[0]
    print(f"Verification: 'stock_history' has {count} rows (expected ~18,750; 15 stocks * ~1250 days).")
    
    cur.execute("SELECT ticker, date, open, high, close, currency FROM stock_history LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'stock_history': {sample}")

In [21]:
# Market list (3 indices)
MARKETS = ['^GSPC', '^DJI', '^IXIC']

# Create table
def create_market_history_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS market_history (
            index_symbol VARCHAR,
            date DATE,
            open DECIMAL,
            high DECIMAL,
            low DECIMAL,
            close DECIMAL,
            volume BIGINT,
            currency VARCHAR,
            PRIMARY KEY (index_symbol, date),
            FOREIGN KEY (index_symbol) REFERENCES market_indices(index_symbol)
        );
    """
    cur.execute(create_sql)
    print("Table 'market_history' created or already exists.")

# Populate table
def populate_market_history():
    history_data = []
    for market in MARKETS:
        ticker = yf.Ticker(market)
        hist = ticker.history(period="5y")  # 5 years of daily data
        for date, row in hist.iterrows():
            history_data.append((
                market, date.strftime('%Y-%m-%d'), float(row["Open"]), float(row["High"]), float(row["Low"]), float(row["Close"]), int(row["Volume"]), ticker.info["currency"]
            ))
        time.sleep(1)  # Avoid rate limits
    execute_values(cur, """
        INSERT INTO market_history (index_symbol, date, open, high, low, close, volume, currency)
        VALUES %s
        ON CONFLICT (index_symbol, date) DO NOTHING
    """, history_data)
    print("Populated 'market_history' table.")

# Verify table
def verify_market_history():
    cur.execute("SELECT COUNT(*) FROM market_history;")
    count = cur.fetchone()[0]
    print(f"Verification: 'market_history' has {count} rows (expected ~3,750; 3 markets * ~1250 days).")
    
    cur.execute("SELECT index_symbol, date, close FROM market_history LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'market_history': {sample}")

In [22]:
def create_sector_top_companies_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS sector_top_companies (
            sector VARCHAR,
            ticker VARCHAR,
            rank INTEGER,
            PRIMARY KEY (sector, ticker),
            FOREIGN KEY (sector) REFERENCES sectors(sector),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'sector_top_companies' created or already exists.")

# Populate table
def populate_sector_top_companies():
    top_data = []
    # Get unique sectors from stocks
    cur.execute("SELECT DISTINCT sector FROM stocks;")
    sectors = [row[0] for row in cur.fetchall()]
    
    for sector in sectors:
        try:
            sector_obj = yf.Sector(sector.lower().replace(' ', '-'))  # e.g., "Technology" -> "technology"
            top_companies = sector_obj.top_companies.head(5)  # Limit to top 5
            for rank, (ticker, _) in enumerate(top_companies.iterrows(), 1):
                # Only include if ticker is in our 15 stocks
                if ticker in STOCKS:
                    top_data.append((sector, ticker, rank))
            time.sleep(1)  # Avoid rate limits
        except Exception as e:
            print(f"Warning: Could not fetch top companies for sector '{sector}': {e}")
    
    execute_values(cur, """
        INSERT INTO sector_top_companies (sector, ticker, rank)
        VALUES %s
        ON CONFLICT (sector, ticker) DO NOTHING
    """, top_data)
    print("Populated 'sector_top_companies' table.")

# Verify table
def verify_sector_top_companies():
    cur.execute("SELECT COUNT(*) FROM sector_top_companies;")
    count = cur.fetchone()[0]
    print(f"Verification: 'sector_top_companies' has {count} rows (expected up to 40; ~8 sectors * 5).")
    
    cur.execute("SELECT * FROM sector_top_companies LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'sector_top_companies': {sample}")

In [23]:
def format_industry_key(industry):
    # Split by hyphen, remove spaces from each part, then rejoin with hyphen
    parts = industry.split(' - ')
    cleaned_parts = [part.replace(' ', '') for part in parts]
    return '-'.join(cleaned_parts)

def create_industry_top_companies_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS industry_top_companies (
            industry VARCHAR,
            ticker VARCHAR,
            category VARCHAR,
            rank INTEGER,
            PRIMARY KEY (industry, ticker),
            FOREIGN KEY (industry) REFERENCES industries(industry),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'industry_top_companies' created or already exists.")

# Populate table
def populate_industry_top_companies():
    top_data = []
    # Get unique industries from stocks
    cur.execute("SELECT DISTINCT industry FROM stocks;")
    industries = [row[0] for row in cur.fetchall()]
    
    for industry in industries:
        try:
            # e.g., "Software—Infrastructure" -> "software-infrastructure"
            industry_obj = yf.Industry(format_industry_key(industry).lower())
            print(industry_obj)
            # e.g., "Software—Infrastructure" -> "software-infrastructure"
            top_companies = industry_obj.top_performing_companies.head(5)  # Limit to top 5
            for rank, (ticker, _) in enumerate(top_companies.iterrows(), 1):
                # Only include if ticker is in our 15 stocks
                if ticker in STOCKS:
                    top_data.append((industry, ticker, "Top Performing", rank))
            time.sleep(1)  # Avoid rate limits
        except Exception as e:
            print(f"Warning: Could not fetch top companies for industry '{industry}': {e}")
    
    execute_values(cur, """
        INSERT INTO industry_top_companies (industry, ticker, category, rank)
        VALUES %s
        ON CONFLICT (industry, ticker) DO NOTHING
    """, top_data)
    print("Populated 'industry_top_companies' table.")

# Verify table
def verify_industry_top_companies():
    cur.execute("SELECT COUNT(*) FROM industry_top_companies;")
    count = cur.fetchone()[0]
    print(f"Verification: 'industry_top_companies' has {count} rows (expected up to 75; ~15 industries * 5).")
    
    cur.execute("SELECT * FROM industry_top_companies LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'industry_top_companies': {sample}")

In [24]:
def create_calendar_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS calendar (
            ticker VARCHAR,
            earnings_date DATE,
            earnings_high DECIMAL,
            earnings_low DECIMAL,
            earnings_average DECIMAL,
            revenue_high BIGINT,
            revenue_low BIGINT,
            revenue_average BIGINT,
            currency VARCHAR,
            PRIMARY KEY (ticker, earnings_date),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'calendar' created or already exists.")

# Populate table
def populate_calendar():
    calendar_data = []
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        cal = ticker.calendar
        if cal is not None and "Earnings Date" in cal:
            earnings_date = cal["Earnings Date"][0] if isinstance(cal["Earnings Date"], list) else cal["Earnings Date"]
            calendar_data.append((
                stock, earnings_date.strftime('%Y-%m-%d'), float(cal.get("Earnings High", 0.0)),
                float(cal.get("Earnings Low", 0.0)), float(cal.get("Earnings Average", 0.0)),
                int(cal.get("Revenue High", 0)), int(cal.get("Revenue Low", 0)),
                int(cal.get("Revenue Average", 0)), ticker.info["currency"]
            ))
        time.sleep(1)
    execute_values(cur, """
        INSERT INTO calendar (ticker, earnings_date, earnings_high, earnings_low, earnings_average,
        revenue_high, revenue_low, revenue_average, currency)
        VALUES %s
        ON CONFLICT (ticker, earnings_date) DO NOTHING
    """, calendar_data)
    print("Populated 'calendar' table.")

# Verify table
def verify_calendar():
    cur.execute("SELECT COUNT(*) FROM calendar;")
    count = cur.fetchone()[0]
    print(f"Verification: 'calendar' has {count} rows (expected 15; 1 per stock).")
    
    cur.execute("SELECT ticker, earnings_date FROM calendar LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'calendar': {sample}")

In [25]:
def create_income_stmt_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS income_stmt (
            ticker VARCHAR,
            date DATE,
            total_revenue BIGINT,
            cost_of_revenue BIGINT,
            gross_profit BIGINT,
            operating_expense BIGINT,
            research_and_development BIGINT,
            selling_general_admin BIGINT,
            operating_income BIGINT,
            ebit BIGINT,
            ebitda BIGINT,
            interest_expense BIGINT,
            interest_income BIGINT,
            other_income_expense BIGINT,
            pretax_income BIGINT,
            tax_provision BIGINT,
            net_income BIGINT,
            basic_eps DECIMAL,
            diluted_eps DECIMAL,
            basic_average_shares BIGINT,
            diluted_average_shares BIGINT,
            currency VARCHAR,
            PRIMARY KEY (ticker, date),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'income_stmt' created or already exists.")

# Populate table
def populate_income_stmt():
    income_data = []
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        stmt = ticker.get_income_stmt(freq="quarterly")
        print(f"Debug: Fetching income statement for {stock}")
        if stmt is None or stmt.empty:
            print(f"Warning: No income statement data for {stock}")
            continue
        for date, row in stmt.items():
            date_str = date.strftime('%Y-%m-%d')
            income_data.append((
                stock, date_str,
                safe_int(row.get("TotalRevenue"), "TotalRevenue", stock, date_str),
                safe_int(row.get("CostOfRevenue"), "CostOfRevenue", stock, date_str),
                safe_int(row.get("GrossProfit"), "GrossProfit", stock, date_str),
                safe_int(row.get("OperatingExpense"), "OperatingExpense", stock, date_str),
                safe_int(row.get("ResearchAndDevelopment"), "ResearchAndDevelopment", stock, date_str),
                safe_int(row.get("SellingGeneralAndAdministration"), "SellingGeneralAndAdministration", stock, date_str),
                safe_int(row.get("OperatingIncome"), "OperatingIncome", stock, date_str),
                safe_int(row.get("EBIT"), "EBIT", stock, date_str),
                safe_int(row.get("EBITDA"), "EBITDA", stock, date_str),
                safe_int(row.get("InterestExpense"), "InterestExpense", stock, date_str),
                safe_int(row.get("InterestIncome"), "InterestIncome", stock, date_str),
                safe_int(row.get("OtherIncomeExpense"), "OtherIncomeExpense", stock, date_str),
                safe_int(row.get("PretaxIncome"), "PretaxIncome", stock, date_str),
                safe_int(row.get("TaxProvision"), "TaxProvision", stock, date_str),
                safe_int(row.get("NetIncome"), "NetIncome", stock, date_str),
                float(row.get("BasicEPS", 0.0)),  # DECIMAL can handle NaN as 0.0
                float(row.get("DilutedEPS", 0.0)),
                safe_int(row.get("BasicAverageShares"), "BasicAverageShares", stock, date_str),
                safe_int(row.get("DilutedAverageShares"), "DilutedAverageShares", stock, date_str),
                "USD"
            ))
        time.sleep(1)  # Avoid rate limits
    print(f"Debug: Prepared {len(income_data)} rows for insertion")
    execute_values(cur, """
        INSERT INTO income_stmt (ticker, date, total_revenue, cost_of_revenue, gross_profit, operating_expense,
        research_and_development, selling_general_admin, operating_income, ebit, ebitda, interest_expense, interest_income,
        other_income_expense, pretax_income, tax_provision, net_income, basic_eps, diluted_eps, basic_average_shares,
        diluted_average_shares, currency)
        VALUES %s
        ON CONFLICT (ticker, date) DO NOTHING
    """, income_data)
    print("Populated 'income_stmt' table.")
    
    
def verify_income_stmt():
    cur.execute("SELECT COUNT(*) FROM income_stmt;")
    count = cur.fetchone()[0]
    print(f"Verification: 'income_stmt' has {count} rows (expected ~300; 15 stocks * ~20 quarters).")
    
    cur.execute("SELECT ticker, date, net_income FROM income_stmt LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'income_stmt': {sample}")

In [26]:
def create_balance_sheet_table():
    create_sql = """
        CREATE TABLE IF NOT EXISTS balance_sheet (
            ticker VARCHAR,
            date DATE,
            total_assets BIGINT,
            current_assets BIGINT,
            non_current_assets BIGINT,
            cash_and_cash_equivalents BIGINT,
            accounts_receivable BIGINT,
            inventory BIGINT,
            property_plant_equipment BIGINT,
            total_liabilities BIGINT,
            current_liabilities BIGINT,
            non_current_liabilities BIGINT,
            accounts_payable BIGINT,
            long_term_debt BIGINT,
            total_debt BIGINT,
            working_capital BIGINT,
            shareholders_equity BIGINT,
            common_stock BIGINT,
            retained_earnings BIGINT,
            currency VARCHAR,
            PRIMARY KEY (ticker, date),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """
    cur.execute(create_sql)
    print("Table 'balance_sheet' created or already exists.")

# Populate table
def populate_balance_sheet():
    balance_data = []
    for stock in STOCKS:
        ticker = yf.Ticker(stock)
        stmt = ticker.get_balance_sheet(freq="quarterly")
        for date, row in stmt.items():
            total_assets = safe_int(row.get("TotalAssets", 0), "TotalAssets", stock, date.strftime('%Y-%m-%d'))
            current_assets = safe_int(row.get("CurrentAssets", 0), "CurrentAssets", stock, date.strftime('%Y-%m-%d'))
            total_liab = safe_int(row.get("TotalLiabilitiesNetMinorityInterest", 0), "TotalLiabilitiesNetMinorityInterest", stock, date.strftime('%Y-%m-%d'))
            current_liab = safe_int(row.get("CurrentLiabilities", 0), "CurrentLiabilities", stock, date.strftime('%Y-%m-%d'))
            non_current_liab = total_liab - current_liab if total_liab is not None and current_liab is not None else None
            non_current_assets = total_assets - current_assets if total_assets is not None and current_assets is not None else None
            balance_data.append((
                stock, date.strftime('%Y-%m-%d'), total_assets, current_assets,non_current_assets ,
                safe_int(row.get("CashAndCashEquivalents", 0), "CashAndCashEquivalents", stock, date.strftime('%Y-%m-%d')), safe_int(row.get("AccountsReceivable", 0), "AccountsReceivable", stock, date.strftime('%Y-%m-%d')),
                safe_int(row.get("Inventory", 0), "Inventory", stock, date.strftime('%Y-%m-%d')), safe_int(row.get("NetPPE", 0), "NetPPE", stock, date.strftime('%Y-%m-%d')), total_liab, current_liab,
                non_current_liab, safe_int(row.get("AccountsPayable", 0), "AccountsPayable", stock, date.strftime('%Y-%m-%d')), safe_int(row.get("LongTermDebt", 0), "LongTermDebt", stock, date.strftime('%Y-%m-%d')),
                safe_int(row.get("TotalDebt", 0), "TotalDebt", stock, date.strftime('%Y-%m-%d')), safe_int(row.get("WorkingCapital", 0), "WorkingCapital", stock, date.strftime('%Y-%m-%d')),
                safe_int(row.get("TotalEquityGrossMinorityInterest", 0), "TotalEquityGrossMinorityInterest", stock, date.strftime('%Y-%m-%d')), safe_int(row.get("CommonStock", 0), "CommonStock", stock, date.strftime('%Y-%m-%d')),
                safe_int(row.get("RetainedEarnings", 0), "RetainedEarnings", stock, date.strftime('%Y-%m-%d')), ticker.info["currency"]
            ))
        time.sleep(1)
    execute_values(cur, """
        INSERT INTO balance_sheet (ticker, date, total_assets, current_assets, non_current_assets,
        cash_and_cash_equivalents, accounts_receivable, inventory, property_plant_equipment, total_liabilities,
        current_liabilities, non_current_liabilities, accounts_payable, long_term_debt, total_debt, working_capital,
        shareholders_equity, common_stock, retained_earnings, currency)
        VALUES %s
        ON CONFLICT (ticker, date) DO NOTHING
    """, balance_data)
    print("Populated 'balance_sheet' table.")

# Verify table
def verify_balance_sheet():
    cur.execute("SELECT COUNT(*) FROM balance_sheet;")
    count = cur.fetchone()[0]
    print(f"Verification: 'balance_sheet' has {count} rows (expected ~300; 15 stocks * ~20 quarters).")
    
    cur.execute("SELECT ticker, date, total_assets FROM balance_sheet LIMIT 1;")
    sample = cur.fetchone()
    print(f"Sample row from 'balance_sheet': {sample}")

In [27]:
from datetime import datetime

def create_watchlists_table():
    cur.execute("""
        CREATE TABLE IF NOT EXISTS watchlists (
            user_id VARCHAR, ticker VARCHAR, added_at TIMESTAMP,
            PRIMARY KEY (user_id, ticker),
            FOREIGN KEY (user_id) REFERENCES users(user_id),
            FOREIGN KEY (ticker) REFERENCES stocks(ticker)
        );
    """)
    print("Table 'watchlists' created or already exists.")

def populate_watchlists():
    watchlist_data = [
        ('user1', 'AAPL', datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        ('user1', 'MSFT', datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        ('user2', 'GOOGL', datetime.now().strftime('%Y-%m-%d %H:%M:%S')),
        ('user3', 'TSLA', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    ]
    print(f"Debug: Prepared {len(watchlist_data)} rows")
    execute_values(cur, """
        INSERT INTO watchlists (user_id, ticker, added_at)
        VALUES %s ON CONFLICT (user_id, ticker) DO NOTHING
    """, watchlist_data)
    print("Populated 'watchlists' table.")

def verify_watchlists():
    cur.execute("SELECT COUNT(*) FROM watchlists;")
    print(f"Verification: 'watchlists' has {cur.fetchone()[0]} rows (expected 4).")
    cur.execute("SELECT user_id, ticker, added_at FROM watchlists LIMIT 1;")
    print(f"Sample row from 'watchlists': {cur.fetchone()}")

In [28]:
# Execute and verify
try:
    print("Starting process...")

    # create_calendar_table()
    # populate_calendar()
    # verify_calendar()

    # create_income_stmt_table()
    # populate_income_stmt()
    # verify_income_stmt()
    
    # create_balance_sheet_table()
    # populate_balance_sheet()
    
    # verify_balance_sheet()
    
    
    create_watchlists_table()
    populate_watchlists()
    verify_watchlists()
    
    conn.commit()
    print("Tables 'users' and 'market_indices' processed successfully!")
except Exception as e:
    conn.rollback()
    print(f"Error occurred: {e}")
finally:
    cur.close()
    conn.close()

Starting process...
Table 'watchlists' created or already exists.
Debug: Prepared 4 rows
Populated 'watchlists' table.
Verification: 'watchlists' has 4 rows (expected 4).
Sample row from 'watchlists': ('user1', 'AAPL', datetime.datetime(2025, 4, 12, 10, 7, 59))
Tables 'users' and 'market_indices' processed successfully!
